In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Hello World
This tutorial/example aims to give an simplistic overview of setting up a passive volume (scattering material to be imaged), detector (active material to record hits), and optimisation loop (to refine the detector to image the passive volume better or more cheaply)

## Volume setup
The volume consists of both passive material to be imaged, and active detectors to record muons hits. We aim to adjust the position and xy-span of the detectors to minimise the cost of the detector whilst improving the precision of the imaging of the passive volume.

<img src="imgs/panel detector.png" width="256">

The above image shows a typical layout, with multiple detection layers above and below the passive volume consisting of layers of material. Each passive layer consists of sub-cubes (voxels, these can be different materials, whose X0 will affect the scattering of muons to deferring extents.. In the detector layers, there can be multiple panels with fixed resolution and efficiency. The xyz position of the panels and their xy extent are the properties to be optimised.

In [2]:
from tomopt.core import *

/Users/giles/anaconda3/envs/tomopt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
from torch import Tensor
from torch.nn import functional as F

import matplotlib.pyplot as plt
import seaborn as sns

/Users/giles/anaconda3/envs/tomopt/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
DEVICE = torch.device('cpu')

Construct a volume by sandwiching passive layers with pairs of detector layers above and below. Detector panels are initialised with specified parameters, but the number of detector panels per layer is fixed. Passive layers will contain voxels of different materials.

In [5]:
import numpy as np
from torch import nn
from tomopt.volume import PassiveLayer, PanelDetectorLayer, DetectorPanel

In [6]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.9
    init_res = 1e4
    n_panels = 4
    layers.append(PanelDetectorLayer(pos='above', lw=lwh[:2], z=1, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.25,0.25,1-(i*(2*size)/n_panels)], init_xy_span=[1.0, 1.0],
                                                      device=DEVICE) for i in range(n_panels)]))
    for z in [0.8,0.7,0.6,0.5,0.4,0.3]:
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=DEVICE))
    layers.append(PanelDetectorLayer(pos='below', lw=lwh[:2], z=0.2, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.25,0.25,0.2-(i*(2*size)/n_panels)], init_xy_span=[1.0, 1.0],
                                                      device=DEVICE) for i in range(n_panels)]))

    return nn.ModuleList(layers) 

We'll feed these stacked layers into a single object for convenience.

In [7]:
from tomopt.volume import Volume

In [56]:
volume = Volume(get_layers())
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([1.]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.9000]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2500, 0.2500]), z=tensor([0.8500]), and xy span tensor([1., 1.]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([0.8000])
    (2): PassiveLayer located at z=tensor([0.7000])
    (3): PassiveLayer located at z=tensor([0.6000])
    (4): PassiveLayer located at z=te

In [57]:
volume.get_cost()

tensor(8., grad_fn=<AddBackward0>)

## Optimisation

Having built our detector and defined where the passive volumes will be, we now want to define the material composition of the passive volume(s) to be imaged, and to optimise our detector.
As its name suggests, `PanelVolumeWrapper` wraps the `Volume` and provides methods to optimise the detector, predict passive volumes, loading and saving of detector configurations, etc.

When building the wrapper, we need to supply it with optimiser definitions for both resolution and efficiency, and a loss function to optimise. Optimisers are the classes in PyTorch which implement gradient descent on specified parameters. For the optimisers, use partial definitions, since the `PanelVolumeWrapper` takes care of instantiating the optimisers to relate to the relevant detector parameters. Although we set some learning rates (`lr`) here, it is difficult to know a priori what are suitable values. When running the optimisation, though, we will use a special callback to set more suitable values.

In [58]:
from functools import partial
from tomopt.optimisation import PanelVolumeWrapper, VoxelX0Loss

In [59]:
wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VoxelX0Loss(target_budget=20, cost_coef=None))  # Loss is precision + budget_coef*cost_coef, balance coef as required or leave as None to automatically balance on first batch

Now we need to define the composition of the passive volume. This is done via a function which is called per passive layer (we have six) and returns the x0 (radiation length [m]) of the material in the voxels in that layer. Below we'll define the passive material to be mostly beryllium but with a single block of lead in the corner of one of the layers.

In [60]:
from tomopt.core import X0

In [61]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> Tensor:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['lead']
    return rad_length

Since we may want to optimise the detector for multiple different passive volumes (or randomly generated volumes), different functions can be passed to the `PanelVolumeWrapper` during optimisation by loading the functions into `PassiveYielder` objects, which loop through the passive volumes. In this tutorial, though, we only have one passive volume.

In [62]:
from tomopt.optimisation import PassiveYielder

In [63]:
passives = PassiveYielder([arb_rad_length])

Now we're in a position to optimise our detector using the `VolumeWrapper.fit` method. The general layout for the optimisation loop is as follows, where `mu_bs` is the number of muons passed through the volume simultaneously and `passive_bs` is the number of passive volumes used per detector update:
1. for epoch in `n_epochs`:
    1. `loss` = 0
    1. for `p`, `passive` in enumerate(`trn_passives`):
        1. load `passive` into passive volume
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. Irradiate volume with `mu_bs` muons
            1. Infer passive volume
        1. Compute loss based on precision and cost, and add to `loss`
        1. if `p`+1 % `passive_bs` == 0:
            1. `loss` = `loss`/`passive_bs`
            1. Backpropagate `loss` and update detector parameters
            1. `loss` = 0
        1. if len(`trn_passives`)-(`p`+1) < `passive_bs`:
            1. Break
            
    1. `val_loss` = 0
    1. for `p`, `passive` in enumerate(`val_passives`):
        1. load `passive` into passive volume
        1. for muon_batch in range(`n_mu_per_volume`//`mu_bs`):
            1. Irradiate volume with `mu_bs` muons
            1. Infer passive volume
            1. Compute loss based on precision and cost, and add to `val_loss`
        1. if len(`val_passives`)-(`p`+1) < `passive_bs`:
            1. Break
    1. `val_loss` = `val_loss`/`p`

I.e. the loss for a single passive can be computed over several iterations of muon irradiation (for computational speed), and the total loss can be accumulated over multiple passives before updating the detector. In our case we only have one passive. We also have the option of evaluating the detector on unseen passives via validation data. For this example, we'll use the same passive for the both training and validation.

Additionally, the optimisation loop has many interjection points for callback classes, which can affect most aspects of the optimisation. A few of the recommended callbacks are listed below:

- `NoMoreNaNs`: Currently in TomOpt, there is a problem which means that sometimes the gradients for some parameters are `NaN` after backpropagation. The `tomopt.optimisation.callbacks.grad_callbacks.NoMoreNaNs` will automatically go through each parameter after backpropagation and replace `NaN` gradients with zeros prior to the optimiser update step.
- `PanelMetricLogger`: It is useful to have an indication of the progress the optimisation, and the current state of the detector. `tomopt.optimisation.callbacks.monitors.MetricLogger` provides real-time progress updates and telemetric data.
- `CostCoefWarmup`: The loss contains a coefficient which is used to balance the error and cost components. This can be set manually or automatically. The loss contains a method to set the coefficient automatically, but it does so using the first volume tested, and so is subject to fluctuations. `tomopt.optimisation.callbacks.loss_callbacks.CostCoefWarmup` is a callback which will run a specified number of warm-up epochs during which the detectors will not be optimised, and instead a more stable cost coefficient will be computed.
- `PanelOptConfig`: A recent addition is the `tomopt.optimisation.callbacks.opt_callbacks.PanelOptConfig` callback. Like the `CostCoefWarmup` it uses a warm-up period to track the gradients received by the detector parameters. The user can then instead specify their desired rate of change rate in physical units, and the learning rates of the optimisers will be overwritten to achieve the specified rates.
- `MuonResampler`: When optimising the detectors, we only want to consider muons which pass through the passive volume, however not all muons are guaranteed to do so. The `tomopt.optimisation.callbacks.data_callbacks.MuonResampler` will automatically regenerate muons which will not pass through the passive volume. Since this relies on "truth" information in the generator, this should not be used for realistic simulations.

In [64]:
from tomopt.optimisation import NoMoreNaNs, PanelMetricLogger, CostCoefWarmup, PanelOptConfig, MuonResampler

In [65]:
from tomopt.optimisation import Callback
from typing import *

In [66]:
class PanelUpdateLimiter(Callback):
    def __init__(self, max_xy_step:Tuple[float,float], max_z_step:float, max_xy_span_step:Tuple[float,float]):
        self.max_xy_step = Tensor(max_xy_step)
        self.max_z_step = Tensor([max_z_step])
        self.max_xy_span_step = Tensor(max_xy_span_step)
        
    def on_backwards_end(self) -> None:
        self.panel_params:List[Dict[str,Tensor]] = []
        for det in self.wrapper.volume.get_detectors():
            for panel in det.panels:
                self.panel_params.append({'xy':panel.xy.detach().clone(),
                                          'z':panel.z.detach().clone(),
                                          'xy_span':panel.xy_span.detach().clone()})
                
    def on_step_end(self) -> None:
        with torch.no_grad():
            panel_idx = 0
            for det in self.wrapper.volume.get_detectors():
                for panel in det.panels:
                    delta = panel.xy-self.panel_params[panel_idx]['xy']
                    panel.xy.data = torch.where(delta.abs() > self.max_xy_step,
                                           self.panel_params[panel_idx]['xy']+(torch.sign(delta)*self.max_xy_step),
                                           panel.xy)
                    delta = panel.z-self.panel_params[panel_idx]['z']
                    panel.z.data = torch.where(delta.abs() > self.max_z_step,
                                           self.panel_params[panel_idx]['z']+(torch.sign(delta)*self.max_z_step),
                                           panel.z)
                    delta = panel.xy_span-self.panel_params[panel_idx]['xy_span']
                    panel.xy_span.data = torch.where(delta.abs() > self.max_xy_span_step,
                                           self.panel_params[panel_idx]['xy_span']+(torch.sign(delta)*self.max_xy_span_step),
                                           panel.xy_span)
                    panel_idx += 1

In [67]:
cb = PanelUpdateLimiter(max_xy_step=(0.01,0.01), max_xy_span_step=(0.01,0.01), max_z_step=0.005)

In [68]:
cb.set_wrapper(wrapper)

In [69]:
cb.on_backwards_end()

In [70]:
cb.panel_params

[{'xy': tensor([0.2500, 0.2500]),
  'z': tensor([1.]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.9500]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.9000]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.8500]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.2000]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.1500]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.1000]),
  'xy_span': tensor([1., 1.])},
 {'xy': tensor([0.2500, 0.2500]),
  'z': tensor([0.0500]),
  'xy_span': tensor([1., 1.])}]

In [71]:
with torch.no_grad():
    for det in wrapper.volume.get_detectors():
        for panel in det.panels:
            panel.xy += 10
            panel.xy_span += 10
            panel.z -= 10

In [72]:
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([10.2500, 10.2500]), z=tensor([11.]), and xy span tensor([11., 11.]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([10.2500, 10.2500]), z=tensor([10.9500]), and xy span tensor([11., 11.]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([10.2500, 10.2500]), z=tensor([10.9000]), and xy span tensor([11., 11.]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([10.2500, 10.2500]), z=tensor([10.8500]), and xy span tensor([11., 11.]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([0.8000])
    (2): PassiveLayer located at z=tensor([0.7000])
    (3): PassiveLayer located at z=tensor([0.6000])
    (4): PassiveL

In [73]:
cb.on_step_end()

In [74]:
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2600, 0.2600]), z=tensor([1.0050]), and xy span tensor([1.0100, 1.0100]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2600, 0.2600]), z=tensor([0.9550]), and xy span tensor([1.0100, 1.0100]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2600, 0.2600]), z=tensor([0.9050]), and xy span tensor([1.0100, 1.0100]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.2600, 0.2600]), z=tensor([0.8550]), and xy span tensor([1.0100, 1.0100]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([0.8000])
    (2): PassiveLayer located at z=tensor([0.7000])
    (3): PassiveLayer located at z=tensor([0.6000])
 

In [ ]:
_ = wrapper.fit(n_epochs=20,
                passive_bs=1,
                n_mu_per_volume=1000,
                mu_bs=100,
                trn_passives=passives,
                val_passives=passives,
                cbs=[CostCoefWarmup(n_warmup=5),
                     PanelOptConfig(n_warmup=5, xy_pos_rate=0.01, z_pos_rate=0.005, xy_span_rate=0.01),
                     MuonResampler(),
                     NoMoreNaNs(),
                     PanelMetricLogger()])

In [ ]:
volume.get_cost()

## Predictions
`VolumeWrapper` also has a `.predict` method to provide predictions per passive volume. For convenience it also returns the true material composition for each prediction. We'll include the `MuonResampler` under the assumption that we have a trigger/classifier that rejects muons that miss the passive volume.

In [ ]:
preds = wrapper.predict(passives,
                        n_mu_per_volume=10000,
                        mu_bs=100,
                        cbs=[MuonResampler()])

`preds` is a list of tuples of predicted x0 and true x0 per passive volume. We can plot these with `tomopt.plotting.predictions.plot_pred_true_x0`.

In [ ]:
from tomopt.plotting import plot_pred_true_x0

In [ ]:
plot_pred_true_x0(*preds[0])

Above we have the predicted and true x0 for ever voxel in the six passive layers. Currently the scale of predictions is very low, due to biases in the prediction method.